In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys, os
utils_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm/src/utils")
sys.path.append(str(utils_path))

In [3]:
import pandas as pd
import numpy as np

from pdf2image import convert_from_path
from PIL import Image

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from langchain_core.documents import Document
# from langchain.vectorstores import FAISS

from utils import generate_random_name

c:\Users\shaur\anaconda3\envs\kg_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Paths

In [4]:
base_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm")
artifacts_path = os.path.join(base_path, "artifacts")
poppler_path = os.path.join(artifacts_path, "libraries/Release-23.08.0-0/poppler-23.08.0/Library/bin")
supp_apps_pdfs_dir_path = os.path.join(artifacts_path, "applications/pdfs")
supp_apps_pngs_dir_path = os.path.join(artifacts_path, "applications/pngs")
supp_apps_chunked_pngs_dir_path = os.path.join(artifacts_path, "applications/chunked_pngs")

## PDF to PNG

In [5]:
example_pdf_path = os.path.join(supp_apps_pdfs_dir_path, "crc_app.pdf")

In [6]:
example_pdf_pages = convert_from_path(pdf_path=example_pdf_path, poppler_path=poppler_path)
example_pdf_name = os.path.splitext(os.path.basename(example_pdf_path))[0]
# c=1
# for page in example_pdf_pages:
#     os.makedirs(supp_apps_pngs_dir_path, exist_ok=True)
#     img_name = f"{example_pdf_name}_{c}.png"
#     page.save(os.path.join(supp_apps_pngs_dir_path, img_name), "PNG")
#     c+=1

In [7]:
elements = partition_pdf(
    filename=example_pdf_path,
    strategy="hi_res",
)

In [8]:
def chunk_pdf_by_titles(pdf_path, title_criteria="heading"):
    
    # Partition the PDF into elements
    elements = partition_pdf(filename=pdf_path)
    
    chunks = chunk_by_title(elements)

    for chunk in chunks:
        print(chunk)
        print("\n\n" + "-"*80)

chunks = chunk_pdf_by_titles(example_pdf_path)

> Pro-Praxis

Senior Care Liability Supplemental “Application”

Additional Information Needed:

e Acord Applications for each line of coverage requested

e 6 years of loss runs from prior insurers currently valued within 60 days of effective date Completed supplemental application for each facility considered for coverage. Most current year financial statement — audited if available

e Resident Agreement


--------------------------------------------------------------------------------
e Licenses

e Facility web site URL

e Any contracts with Medical Director, outside wound care team, management company or other contracted healthcare providers

e — If current coverage is Claims Made, provide copy of current policy

Applicant Information

Named Insured: Senior Living LLC Mailing Address|1234 Northside Blvd

Tax Status / Corporate Structure

EE] For-Profit [Not-for-Profit Religious Affiliated? [Yes [No


--------------------------------------------------------------------------------
O P

In [9]:
documents = []
for element in elements:
    if len(element.text)<20: continue
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

# Chunk images and update documents with chunked img paths
c=1
for document in documents:
    chunk_page_no = document.metadata['page_number']
    matched_page_img_name = f"{example_pdf_name}_{chunk_page_no}.png"

    matched_page_img_path = os.path.join(supp_apps_pngs_dir_path, matched_page_img_name)
    matched_page_img = Image.open(matched_page_img_path)

    chunked_coords = document.metadata['coordinates']['points']
    chunked_region = (chunked_coords[0][0], chunked_coords[0][1], chunked_coords[2][0], chunked_coords[2][1])
    chunked_img =  matched_page_img.crop(chunked_region)

    os.makedirs(supp_apps_chunked_pngs_dir_path, exist_ok=True)
    chunked_img_name = f"{example_pdf_name}_{c}.png"
    chunked_img_path = os.path.join(supp_apps_chunked_pngs_dir_path, chunked_img_name)
    chunked_img.save(chunked_img_path)
    document.metadata['chunked_img_path'] = chunked_img_path

    c+=1
